In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk, stability_function

rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()

In [ ]:
def RRK_pos(rkm, dt, f, w0=[1.,0], t_final=1., relaxation=True, 
        rescale_step=True, debug=False, gammatol=10.1, print_gamma=False,
        one_step=False):
    """
    Relaxation Runge-Kutta method implementation.
    
    Options:
    
        rkm: Base Runge-Kutta method, in Nodepy format
        dt: time step size
        f: RHS of ODE system
        w0: Initial data
        t_final: final solution time
        relaxation: if True, use relaxation method.  Otherwise, use vanilla RK method.
        rescale_step: if True, new time step is t_n + \gamma dt
        debug: output some additional diagnostics
        gammatol: Fail if abs(1-gamma) exceeds this value
        
    """
    w = np.array(w0)
    t = 0
    # We pre-allocate extra space because if rescale_step==True then
    # we don't know exactly how many steps we will take.
    ww = np.zeros([len(w0),int((t_final-t)/dt*2.5)+10000])
    ww[:,0] = w.copy()
    tt = [t]
    ii = 0
    s = len(rkm)
    b = rkm.b
    y = np.zeros((s,len(w0)))
    max_gammam1 = 0.
    gams = []
    
    while t < t_final:
        if t + dt >= t_final:
            dt = t_final - t # Hit final time exactly
        
        for i in range(s):
            y[i,:] = w.copy()
            for j in range(i):
                y[i,:] += rkm.A[i,j]*dt*f(y[j,:])
                
        F = np.array([f(y[i,:]) for i in range(s)])
        dw = dt*sum([b[j]*F[j] for j in range(s)])
        
        eps = 1.e-10
        gam = 1.
        if relaxation:
            wnp1 = w + dw
            neg = np.nonzero(wnp1<-eps)[0]
            if len(neg)>0:
                for ind in neg:
                    #print(ind,w[ind],dw[ind])
                    gam_ind = w[ind]/(w[ind]-wnp1[ind])
                    gam = min(gam, gam_ind)
            if gam!=1: print('gam: ',gam)
            w = w + gam*dw

        else:  # Use standard RK method
            w = w + dw
           
        if print_gamma:
            print(gam)
        
        if np.abs(gam-1.) > gammatol:
            print(gam)
            raise Exception("The time step is probably too large.")
            
        if gam == 0:
            raise Exception("gamma=0")
        
        if (t+dt < t_final) and rescale_step:
            t += gam*dt
        else:
            t += dt
        ii += 1
        tt.append(t)
        ww[:,ii] = w.copy()
        if debug:
            gm1 = np.abs(1.-gam)
            max_gammam1 = max(max_gammam1,gm1)
            gams.append(gam)
            
        if one_step:
            return w, gam
            
    if debug:
        print(max_gammam1)
        return tt, ww[:, :ii+1], np.array(gams)
    else:
        return tt, ww[:,:ii+1]

In [ ]:
def f2(u):
    return -u**2

In [ ]:
u0 = np.array([1.])
tt, uu = RRK_pos(ssp2,dt=1.6,f=f2,w0=u0,t_final=10.,relaxation=True)

uu=uu.squeeze()
plt.plot(tt,uu,'-o')

In [ ]:
N=50
x = np.linspace(0,1,N)
dx = x[1]-x[0]

def f_heat(u):
    du = np.zeros_like(u)
    du[1:-1] = (u[2:]-2*u[1:-1]+u[:-2])
    du[0] = -2*u[0]+u[1]
    du[-1] = u[-2]-2*u[-1]
    return du/dx**2

u0 = (x>0.5)+0.1
dt = 0.7*dx**2

tt, uu = RRK_pos(ssp3,dt=dt,f=f_heat,w0=u0,t_final=1.1,relaxation=True)
print(np.min(uu))

In [ ]:
plt.pcolor(uu[:,::100])
plt.colorbar()

In [ ]:
from matplotlib import animation
from clawpack.visclaw.JSAnimation import IPython_display

fig = plt.figure()
line, = plt.plot([],[])
plt.ylim(0,2)
plt.xlim(0,1)

def plot_frame(j):
    line.set_data(x,uu[:,10*j]);
    
animation.FuncAnimation(fig,plot_frame)

# Advection
## Centered difference

In [ ]:
N=50
x = np.linspace(0,1,N)
dx = x[1]-x[0]

def f_centered_advection(u):
    du = np.zeros_like(u)
    du[1:-1] = (u[2:]-u[:-2])
    du[0] = u[1]-u[-1]
    du[-1] = u[0]-u[-2]
    return -du/(2*dx)

u0 = (x<0.5)+0.5
u0 = np.sin(2*np.pi*x)+1.

dt = 2.5*dx

tt, uu = RRK_pos_DIRK(trbdf,dt=dt,f=f_centered_advection,w0=u0,t_final=10.,relaxation=True)
print(np.min(uu))

In [ ]:
plt.pcolor(uu[:,::1])
plt.colorbar()

In [ ]:
fig = plt.figure()
line, = plt.plot(x,uu[:,0])
plt.xlim(0,1)
skip = 5
def plot_frame(j):
    line.set_data(x,uu[:,skip*j]);
    
animation.FuncAnimation(fig,plot_frame,frames=int(uu.shape[1]/skip))

In [ ]:
def f_upwind_advection(u):
    du = np.zeros_like(u)
    du[1:] = u[1:]-u[:-1]
    du[0] = u[0]-u[-1]
    return -du/dx

u0 = (x<0.5)+0.1
#u0 = np.sin(2*np.pi*x)+1.
dt = 20.*dx

tt, uu = RRK_pos_DIRK(trbdf,dt=dt,f=f_centered_advection,w0=u0,t_final=10.,relaxation=True)
print(np.min(uu))

In [ ]:
plt.pcolor(uu[:,::1])
plt.colorbar()

In [ ]:
fig = plt.figure()
line, = plt.plot(x,uu[:,0])
plt.xlim(0,1)
plt.ylim(0,2)
skip = 1
def plot_frame(j):
    line.set_data(x,uu[:,skip*j]);
    
animation.FuncAnimation(fig,plot_frame,frames=int(uu.shape[1]/skip))

In [ ]:
from scipy.optimize import fsolve

def RRK_pos_DIRK(rkm, dt, f, w0=[1.,0], t_final=1., relaxation=True, 
        rescale_step=True, debug=False, gammatol=10.1, print_gamma=False,
        one_step=False):
    """
    Relaxation Runge-Kutta method implementation.
    
    Options:
    
        rkm: Base Runge-Kutta method, in Nodepy format
        dt: time step size
        f: RHS of ODE system
        w0: Initial data
        t_final: final solution time
        relaxation: if True, use relaxation method.  Otherwise, use vanilla RK method.
        rescale_step: if True, new time step is t_n + \gamma dt
        debug: output some additional diagnostics
        gammatol: Fail if abs(1-gamma) exceeds this value
        
    """
    w = np.array(w0)
    t = 0
    # We pre-allocate extra space because if rescale_step==True then
    # we don't know exactly how many steps we will take.
    ww = np.zeros([len(w0),int((t_final-t)/dt*2.5)+10000])
    ww[:,0] = w.copy()
    tt = [t]
    ii = 0
    s = len(rkm)
    b = rkm.b
    y = np.zeros((s,len(w0)))
    max_gammam1 = 0.
    gams = []
    
    while t < t_final:
        if t + dt >= t_final:
            dt = t_final - t # Hit final time exactly
        
        for i in range(s):
            stageeq = lambda Y: (Y-w-dt*sum([rkm.A[i,j]*f(y[j,:]) for j in range(i)]) \
                                 - dt*rkm.A[i,i]*f(Y)).squeeze()
            nexty, info, ier, mesg = fsolve(stageeq,w,full_output=1)
            if ier != 1: print(mesg)
            y[i,:] = nexty.copy()
  
        F = np.array([f(y[i,:]) for i in range(s)])
        dw = dt*sum([b[j]*F[j] for j in range(s)])
        
        eps = 1.e-10
        gam = 1.
        if relaxation:
            wnp1 = w + dw
            neg = np.nonzero(wnp1<-eps)[0]
            if len(neg)>0:
                for ind in neg:
                    #print(ind,w[ind],dw[ind])
                    gam_ind = w[ind]/(w[ind]-wnp1[ind])
                    gam = min(gam, gam_ind)
            if gam!=1: print('gam: ',gam)
            w = w + gam*dw

        else:  # Use standard RK method
            w = w + dw
           
        if print_gamma:
            print(gam)
        
        if np.abs(gam-1.) > gammatol:
            print(gam)
            raise Exception("The time step is probably too large.")
        
        if gam == 0:
            raise Exception("gamma=0")
            
        if (t+dt < t_final) and rescale_step:
            t += gam*dt
        else:
            t += dt
        ii += 1
        tt.append(t)
        ww[:,ii] = w.copy()
        if debug:
            gm1 = np.abs(1.-gam)
            max_gammam1 = max(max_gammam1,gm1)
            gams.append(gam)
            
        if one_step:
            return w, gam
            
    if debug:
        print(max_gammam1)
        return tt, ww[:, :ii+1], np.array(gams)
    else:
        return tt, ww[:,:ii+1]

# Back to the heat equation
## Trapezoidal IRK without relaxation

In [ ]:
N=500
x = np.linspace(0,1,N)
dx = x[1]-x[0]

u0 = 1.*(x>0.5)
dt = 1.*dx

tt, uu = RRK_pos_DIRK(irk2,dt=dt,f=f_heat,w0=u0,t_final=0.1,relaxation=False)
print(np.min(uu))

plt.pcolor(uu[:,::1])
plt.colorbar()

In [ ]:
from matplotlib import animation
from clawpack.visclaw.JSAnimation import IPython_display

fig = plt.figure()
line, = plt.plot([],[])
plt.ylim(-0.5,2)
plt.xlim(0,1)

def plot_frame(j):
    line.set_data(x,uu[:,j]);
    
animation.FuncAnimation(fig,plot_frame,frames=uu.shape[1])

## Trapezoidal IRK with relaxation

In [ ]:
u0 = 1.*(x>0.5)
dt = 1.*dx

tt, uu = RRK_pos_DIRK(irk2,dt=dt,f=f_heat,w0=u0,t_final=0.1,relaxation=True)
print(np.min(uu))

plt.pcolor(uu[:,::1])
plt.colorbar()

In [ ]:
fig = plt.figure()
line, = plt.plot([],[])
plt.ylim(-0.5,2)
plt.xlim(0,1)

def plot_frame(j):
    line.set_data(x,uu[:,j]);
    
animation.FuncAnimation(fig,plot_frame,frames=uu.shape[1])

## TR-BDF2 for reference

In [ ]:
u0 = 1.*(x>0.5)
dt = 1.*dx

tt, uu = RRK_pos_DIRK(trbdf,dt=dt,f=f_heat,w0=u0,t_final=0.1,relaxation=True)
print(np.min(uu))

fig = plt.figure()
line, = plt.plot([],[])
plt.ylim(-0.5,2)
plt.xlim(0,1)

def plot_frame(j):
    line.set_data(x,uu[:,j]);
    
animation.FuncAnimation(fig,plot_frame,frames=uu.shape[1])

Add fully-discrete convergence results.

# Atmospheric pollution model

In [ ]:
# Atmospheric pollution model from Hundsdorfer & Verwer

def k1(t):
    th = t/3600  # Time in hours
    thbar = th - 24*(th//24)
    if thbar>4 and thbar<20:
        sec = (np.sin(np.pi/16 *(thbar-4)))**0.2
        return 1.e-5*np.exp(7.*sec)
    else:
        return 1.e-40

k3 = 1.e-16
k2 = 1.e5
sigma2 = 1.e6

def f_pollution(u,t):
    du = np.zeros(4)
    du[0] = k1(t)*u[2] - k2*u[0]
    du[1] = k1(t)*u[2] - k3*u[1]*u[3] + sigma2
    du[2] = k3*u[1]*u[3]-k1(t)*u[2]
    du[3] = k2*u[0] - k3*u[1]*u[3]
    return du

In [ ]:
from scipy.optimize import fsolve

def RRK_pos_DIRK_NA(rkm, dt, f, w0=[1.,0], t_final=1., relaxation=True, 
        rescale_step=True, debug=False, gammatol=10.1, print_gamma=False,
        one_step=False):
    """
    Relaxation Runge-Kutta method implementation.
    
    Options:
    
        rkm: Base Runge-Kutta method, in Nodepy format
        dt: time step size
        f: RHS of ODE system
        w0: Initial data
        t_final: final solution time
        relaxation: if True, use relaxation method.  Otherwise, use vanilla RK method.
        rescale_step: if True, new time step is t_n + \gamma dt
        debug: output some additional diagnostics
        gammatol: Fail if abs(1-gamma) exceeds this value
        
    """
    w = np.array(w0)
    t = 0
    # We pre-allocate extra space because if rescale_step==True then
    # we don't know exactly how many steps we will take.
    ww = np.zeros([len(w0),int((t_final-t)/dt*2.5)+10000])
    ww[:,0] = w.copy()
    tt = [t]
    ii = 0
    s = len(rkm)
    b = rkm.b
    c = rkm.c
    y = np.zeros((s,len(w0)))
    max_gammam1 = 0.
    gams = []
    
    while t < t_final:
        if t + dt >= t_final:
            dt = t_final - t # Hit final time exactly
        
        for i in range(s):
            stageeq = lambda Y: (Y-w-dt*sum([rkm.A[i,j]*f(y[j,:],t+c[j]*dt) for j in range(i)]) \
                                 - dt*rkm.A[i,i]*f(Y,t+c[i]*dt)).squeeze()
            nexty, info, ier, mesg = fsolve(stageeq,w,full_output=1)
            if ier != 1: print(mesg)
            y[i,:] = nexty.copy()
  
        F = np.array([f(y[i,:],t+c[i]*dt) for i in range(s)])
        dw = dt*sum([b[j]*F[j] for j in range(s)])
        
        eps = 1.e-10
        gam = 1.
        if relaxation:
            wnp1 = w + dw
            neg = np.nonzero(wnp1<-eps)[0]
            if len(neg)>0:
                for ind in neg:
                    #print(ind,w[ind],dw[ind])
                    gam_ind = w[ind]/(w[ind]-wnp1[ind])
                    gam = min(gam, gam_ind)
            if gam!=1: print('gam: ',gam)
            w = w + gam*dw

        else:  # Use standard RK method
            w = w + dw
           
        if print_gamma:
            print(gam)
        
        if np.abs(gam-1.) > gammatol:
            print(gam)
            raise Exception("The time step is probably too large.")
        
        if (t+dt < t_final) and rescale_step:
            t += gam*dt
        else:
            t += dt
        ii += 1
        tt.append(t)
        ww[:,ii] = w.copy()
        if debug:
            gm1 = np.abs(1.-gam)
            max_gammam1 = max(max_gammam1,gm1)
            gams.append(gam)
            
        if one_step:
            return w, gam
            
    if debug:
        print(max_gammam1)
        return tt, ww[:, :ii+1], np.array(gams)
    else:
        return tt, ww[:,:ii+1]

In [ ]:
t_final = 3600*24*6
dt = 3600

tref, uref = RRK_pos_DIRK_NA(trbdf,dt=dt/128,f=f_pollution,w0=u0,
                             t_final=t_final,relaxation=False)
plt.plot(np.array(tref)/3600/24,uref[0,:],'-k');
plt.xlim(0,6);

In [ ]:
u0 = np.array([0,1.3e8,5.e11,8.e11])

tt, uu = RRK_pos_DIRK_NA(irk2,dt=dt,f=f_pollution,w0=u0,t_final=t_final,relaxation=False)
plt.plot(np.array(tref)/3600/24,uref[0,:],'-k');
plt.plot(np.array(tt)/3600/24,uu[0,:]);
plt.xlim(0,6);
plt.title('Trapezoidal without relaxation');
skip = int((len(uref[0,:])-1)/(len(uu[0,:])-1))
error = dt*np.sum(np.abs(uu[0,:]-uref[0,::skip]))
print(error/1.e6)

In [ ]:
u0 = np.array([0,1.3e8,5.e11,8.e11])

def solve_pollution(rkm,dt,relax=True):
    tt, uu = RRK_pos_DIRK_NA(rkm,dt=dt,f=f_pollution,w0=u0,t_final=t_final,relaxation=relax)
    plt.plot(np.array(tref)/3600/24,uref[0,:],'-k');
    plt.plot(np.array(tt)/3600/24,uu[0,:]);
    plt.xlim(0,6);
    if relax:
        plt.title(rkm.shortname+' with relaxation');
    else:
        plt.title(rkm.shortname+' without relaxation');
    skip = int((len(uref[0,:])-1)/(len(uu[0,:])-1))
    error = dt*np.sum(np.abs(uu[0,:]-uref[0,::skip]))
    print(error/1.e6)
    
solve_pollution(irk2,dt,relax=False)

In [ ]:
solve_pollution(irk2,dt,relax=True)

In [ ]:
solve_pollution(trbdf,dt,relax=True)

In [ ]:
solve_pollution(be,dt,relax=True)

In [ ]:
solve_pollution(be,dt/16,relax=True)